Importar libraries

In [342]:
import cv2 as cv
# import cv2.dnn as dnn
# import imutils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import seaborn as sns 
from kornia.color import rgb_to_ycbcr
from typing import Dict, List, Tuple
import os, random, math
from statistics import mean
# import sys
# import glob, buscar ficheros en recursivo dentro uno directorio
from pathlib import Path
import torch, torchvision
import torch.nn as nn 
import torch.optim as optim
from torchvision import transforms
import torch.utils.data
from torch.utils.data import DataLoader, Dataset, random_split
from torch.utils.tensorboard import SummaryWriter
# from torchdata.datapipes.iter import IterableWrapper
import torchmetrics as metrics
from torchinfo import summary
from timeit import default_timer as timer
from tqdm.auto import tqdm
# from IPython import display
from PIL import Image

# Primero proba entrenar los datos desde fondo y integrar transfer learning mas tarde
# Usa NAFnet como transfer learning
# Cuidarse a las ubicaciones de los hyperparameters
# Incluye diversos opciones de resolucion como 1280 x 720, 1920 x 1080, 2560 x 1440, 3840 x 2160? , o tamano original, compatible con jpg, png, pdf, video?
# Incluye opciones de clarificacion por los images por usar opencv? (Utilisable solo despues de entrenamiento y convertir vuelta a jpg, etc.)

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"
data_directory = r'D:\Documents\ImageEnhancer\ImageData'
output_directory = r'D:\Documents\ImageEnhancer\PredictionOutput'
sample_directory = r'D:\Documents\ImageEnhancer\ForSample'

# torch.manual_seed(random_seed)
# torch.cuda.manual_seed(random_seed)

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:100"

In [343]:
# pre_trained_model = dnn.readNet(model='', config='', framework='torch')
# dnn.readNetFromTorch
# dnn.createTorchImporter

# Cargar los COCO class names
# with open('object_detection_classes_coco.txt', 'r') as f:
#    class_names = f.read().split('\n')
# !nvidia-smi

Definir valores del turbacion

In [344]:
"""def blur(photo):
    blurred = cv.GaussianBlur(photo, (5, 5), cv.BORDER_DEFAULT)
    return blurred"""

'def blur(photo):\n    blurred = cv.GaussianBlur(photo, (5, 5), cv.BORDER_DEFAULT)\n    return blurred'

Reiniciar ubicacion de folder

In [345]:
os.chdir("/")
os.chdir("Documents/ImageEnhancer")
print(os.getcwd())

d:\Documents\ImageEnhancer


Aplicar turbaciones y guardarles a cada folder turbada / blurred

In [346]:
"""os.chdir(train_directory)
for train_photo in os.listdir(train_directory):
    img = cv.imread(train_photo)
    if img is not None:
        print("Image read")
    else:
        print("Not read")
    blurred = blur(img)
    # Guardarlo a folder BlurredTrain
    cv.imwrite(os.path.join(blurred_train_directory, train_photo), blurred)
    print("Saved {train_photo} to BlurredTrain")
    
os.chdir(test_directory)
for test_photo in os.listdir(test_directory):
    img = cv.imread(test_photo)
    if img is not None:
        print("Image read")
    else:
        print("Not read")
    blurred = blur(img)
    # Guardarlo a portfolio BlurredTest
    cv.imwrite(os.path.join(blurred_test_directory, test_photo), blurred)
    print("Saved {test_photo} to BlurredTest")"""


'os.chdir(train_directory)\nfor train_photo in os.listdir(train_directory):\n    img = cv.imread(train_photo)\n    if img is not None:\n        print("Image read")\n    else:\n        print("Not read")\n    blurred = blur(img)\n    # Guardarlo a folder BlurredTrain\n    cv.imwrite(os.path.join(blurred_train_directory, train_photo), blurred)\n    print("Saved {train_photo} to BlurredTrain")\n    \nos.chdir(test_directory)\nfor test_photo in os.listdir(test_directory):\n    img = cv.imread(test_photo)\n    if img is not None:\n        print("Image read")\n    else:\n        print("Not read")\n    blurred = blur(img)\n    # Guardarlo a portfolio BlurredTest\n    cv.imwrite(os.path.join(blurred_test_directory, test_photo), blurred)\n    print("Saved {test_photo} to BlurredTest")'

Distribuir los canales del RGB verso YCbCr

In [347]:
def channelsplit_output(input):
  input = rgb_to_ycbcr(input)
  y, u, v = input.chunk(dim=-3, chunks=3)
  return y, u, v

# Pude seran util? Ajuste contrast para producir images mas claro
# Porque cv no es compatible con pytorch, pude ser util despues de prediccion / traversar de modelo
def he_hsv(input):
    img_hsv = cv.cvtColor(input, cv.COLOR_RGB2HSV)

    # Histogram equalisation on the V-channel
    img_hsv[:, :, 2] = cv.equalizeHist(img_hsv[:, :, 2])

    # convert image back from HSV to RGB
    image_hsv = cv.cvtColor(img_hsv, cv.COLOR_HSV2RGB)
    
    return image_hsv

def clahe_hsv(img):
    hsv_img = cv.cvtColor(img, cv.COLOR_BGR2HSV)

    h, s, v = hsv_img[:,:,0], hsv_img[:,:,1], hsv_img[:,:,2]
    
    # Puede ajuste hyperparameters aqui
    clahe = cv.createCLAHE(clipLimit = 15.0, tileGridSize = (20,20))
    v = clahe.apply(v)

    hsv_img = np.dstack((h,s,v))

    rgb = cv.cvtColor(hsv_img, cv.COLOR_HSV2RGB)
    
    return rgb

Dataloader Class

In [348]:
# Como extractar el y channel y reajuntar despues asi? 
upscale_factor = 4
resize_size = 448

class ImageFolderCustom(Dataset):
    def __init__(self,
                 target_dir: str,
                 resize_size: int,
                 upscale_factor: int,) -> None:
        # Get image paths
        self.paths = list(Path(target_dir).glob("*.jpg"))

        self.hr_transform = transforms.Compose([
            transforms.Resize(size=(resize_size, resize_size), interpolation=Image.BICUBIC),
            # transforms.TrivialAugmentWide(num_magnitude_bins=31),
            transforms.ToTensor(),
        ])

        self.lr_transform = transforms.Compose([
            transforms.Resize(size=(resize_size // upscale_factor, resize_size // upscale_factor), interpolation=Image.BICUBIC),
            # transforms.TrivialAugmentWide(num_magnitude_bins=31),
            transforms.ToTensor(),
        ])

    # 4. Create function to load images
    def load_image(self, index: int) -> Image.Image:
        # Open image via path and returns it
        image_path = self.paths[index]
        return Image.open(image_path)
    
    # 5. Overwrite __len__() method
    def __len__(self) -> int:
        # Returns total no. of samples
        return len(self.paths)
    
    # 6. Overwrite __getitem__() method
    def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
        # Returns one sample of data, data and label (X, y)
        img = self.load_image(index)
        img_path  = self.paths[index].name #  expects path in data_folder/class_name/image.jpeg
        # High res
        img_hr = self.hr_transform(img)
        # Low res
        img_lr = self.lr_transform(img)


        return {"path": img_path, "hr": img_hr, "lr": img_lr,}


Custom datasets

In [349]:
image_data = ImageFolderCustom(target_dir=data_directory,
                                resize_size=resize_size,
                                upscale_factor=upscale_factor)

train_data, test_data = random_split(image_data, [0.8, 0.2])

Dataloaders

In [350]:
# Averiguar otra vez si hace falta el shuffle o no, Como comparar el train y el blurred train si es shuffled?

BATCH_SIZE = 32
num_workers = os.cpu_count() // 4

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              num_workers=0,
                              shuffle=True,
                              pin_memory=True)

train_batches = math.ceil(len(train_data) // BATCH_SIZE)

test_dataloader = DataLoader(dataset=test_data,
                              batch_size=BATCH_SIZE,
                              num_workers=0,
                              shuffle=False,
                              pin_memory=True)

test_batches = math.ceil(len(test_data) // BATCH_SIZE)

Model Class

In [351]:
# Como convertir desde calidad baja verso calidad alta?
# Primera sigue a los ejemplos, luego usar los que te hizo convertiran con turbacion
# Como compresar hidden units verso el siguiente layer?
# Los hyperparameters beneficie de poner en multiples de 8
class ResolutionEnhancer(nn.Module):
    def __init__(self,
                 input_channels: int,
                 hidden_units: int,
                 output_channels: int,
                 upscale_factor: int,):
        
        super().__init__()

        """self.weights = nn.Parameter(torch.randn(1, 
                                                requires_grad=True,
                                                dtype=torch.float))
        self.bias = nn.Parameter(torch.randn(1, 
                                             requires_grad=True,
                                             dtype=torch.float))"""
        # filter_1 = 9 try 5
        # filter_2 = 3 try 1
        # filter_3 = 5 try 3

        padding = 0
        stride = 1

        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_channels,
                      out_channels=hidden_units,
                      kernel_size=1,
                      stride=stride,
                      padding=padding),
            nn.GELU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=1,
                      stride=stride,
                      padding=padding),
        )

        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=(hidden_units // 2), # 1/2 aqui?
                      kernel_size=1,
                      stride=stride,
                      padding=padding),
            nn.GELU(),
            nn.Conv2d(in_channels=(hidden_units // 2),
                      out_channels=(hidden_units // 2),
                      kernel_size=1,
                      stride=stride,
                      padding=padding),
            nn.GELU(),
        )

        # El modelo debe reconstruir el image verso resolucion alta aqui
        # Va a ver como calcular in_channels y out_channels en el documentacion ?
        self.output_layer = nn.Sequential(
            nn.Conv2d(in_channels=(hidden_units // 2), # Ajuste aqui para arreglar los problemas de shape?
                      out_channels=output_channels * (upscale_factor ** 2),
                      kernel_size=1,
                      stride=stride,
                      padding=padding),
            # nn.BatchNorm2d(input_channels),
            # nn.PixelShuffle(upscale_factor=upscale_factor),
        )
    
    def forward(self, input: torch.Tensor):
        # debe devolver output layer, pude ser multiples
        return nn.functional.pixel_shuffle(input = self.output_layer(self.conv_block_2(self.conv_block_1(input))), upscale_factor=upscale_factor)

Upscale image

In [352]:
def upscale_image(model: torch.nn.Module, 
                  img: torch.Tensor,
                  upscale_factor: int,):
    
    """Predict the result based on input image and restore the image as RGB."""
    lr_size = resize_size // upscale_factor
    y_channel_transform = transforms.Compose([transforms.ToPILImage()])
    aux_channel_transform = transforms.Compose([transforms.Resize(size=(lr_size * upscale_factor, lr_size * upscale_factor), interpolation=Image.BICUBIC),
                                               transforms.ToPILImage()])
    output_transform = transforms.Compose([transforms.ToTensor()])

    # print(img.shape)
    # torch.squeeze(img, 1)

    # Si el image es grayscale?
    if img.shape[1] == 1:
        output_img = model(img)
        
    else:
        y, u, v = channelsplit_output(img)

        # Restore the image in RGB color space.
        # print(f"Y channel: {y.shape}")
        y_model = model(y)
        print(y_model.shape)
        y_channel_output = y_channel_transform(torch.squeeze(torch.permute(y_model, (1, 0, 2, 3)), 1)) # ? 
        print(f"Y channel after model: {y_channel_output.shape}")
        u_channel_output = aux_channel_transform(torch.squeeze(torch.permute(u, (1, 0, 2, 3)), 1))
        print(f"U channel after resize: {u_channel_output.shape}")
        v_channel_output = aux_channel_transform(torch.squeeze(torch.permute(v, (1, 0, 2, 3)), 1))
        print(f"V channel after resize: {v_channel_output.shape}")
    
        # Debemos leer el image primera? O es problema de dispositivo?
        remerged = Image.merge(mode='YCbCr', bands=(y_channel_output, u_channel_output, v_channel_output)).convert('RGB')
    
        output_img = output_transform(remerged).unsqueeze(dim=0)
        
        # print("Has logrado aqui")
        
    return output_img


Definir hiperparametros del entrenamiento

In [353]:
learn_rate = 0.01

# adam: decay of first order momentum of gradient
b1 = 0.5

# adam: decay of second order momentum of gradient
b2 = 0.999

# epoch from which to start lr decay
decay_epoch = 20

upscale_factor = 4

In [354]:
def print_train_time(start: float,
                     end: float,
                     device: torch.device = None):
    total_time = end - start
    print(f"Train time on {device}: {total_time: .3f} seconds")

    return total_time

# random_tensor = torch.rand(size=(3, 224, 224)) # height, width, colour channels for images

"""tensor = torch.tensor([3.0, 6.0, 9.0],
                      dtype=None,
                      device="gpu",
                      requires_grad=False)"""

# Se conoce tambien a criterion 
loss_function = nn.MSELoss()

model = ResolutionEnhancer(input_channels=1,
                           hidden_units=64,
                           output_channels=1,
                           upscale_factor=upscale_factor).to(device)
# print(model)

optimiser = optim.Adam(params=model.parameters(),
                       lr=learn_rate,
                       betas=(b1, b2))

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimiser)

psnr = metrics.PeakSignalNoiseRatio().to(device)
ssim = metrics.StructuralSimilarityIndexMeasure().to(device)

# blob = dnn.blobFromImage(image=img, scalefactor=0.01, size=(224, 224), mean=.to(device)(104, 117, 123))

# set the input blob for the neural network
# model.setInput(blob)
# forward pass image blob through the model
# outputs = model.forward()
# out = next(iter(train_dataloader))


Hace los pasos de entrenacion en funcionas

In [355]:
def train_step(model: torch.nn.Module,
               dataloader_iter: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimiser: torch.optim.Optimizer,
               device: torch.device = device):

    # Devuelve el medio de los valores de un corre
    train_results = {"PSNR": Avg_PSNR,
                     "SSIM": Avg_SSIM,
                     "train_loss": train_loss}

    PSNR, SSIM = [], []
    train_loss = 0

    
    # Poner el modelo en modo del entrenamiento
    model.train()

    for i in range(train_batches):

        try:
            img = next(dataloader_iter)

        except StopIteration:
            dataloader_iter = iter(train_dataloader)
            img = next(dataloader_iter)


        lr_img = img["lr"]
        hr_img = img["hr"]


        # Pasar los datos a modelo al frente en utilisaban el funcion forward
        train_pred = upscale_image(model=model,
                                   img=lr_img,
                                   upscale_factor=upscale_factor)

        # Busca forma de compararles con en el image original / alta calidad
        PSNR.append(psnr(hr_img, train_pred))
        SSIM.append(ssim(hr_img, train_pred))

        # Calcular losses/danos (Differencia entre predicion y actual data), en este caso diferencia entre PSNR original y predicion
        loss = loss_fn(hr_img, train_pred)
        train_loss += loss.item()

        # Poner el optimiser a cero
        optimiser.zero_grad()

        # Loss atras?
        torch.Tensor.backward(loss)

        # Hacer progreso al optimiser
        optimiser.step()
        
    
    # Tomar loss average por el lungo del dataloader
    train_loss /= len(dataloader_iter)
    Avg_PSNR = mean(PSNR)
    Avg_SSIM = mean(SSIM)

    return train_results


def test_step(model: torch.nn.Module,
              dataloader_iter: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device = device):
    
    # Devuelve el medio de los valores de un corre
    test_results = {"PSNR": Avg_PSNR,
                    "SSIM": Avg_SSIM,
                    "test_loss": test_loss}
    
    test_loss = 0
    PSNR, SSIM = [], []

    # Poner en modo eval
    model.eval()

    # Iniciar modo inference context manager
    with torch.inference_mode():
        for i in range(test_batches):
            try:
                img = next(dataloader_iter)

            except StopIteration:
                dataloader_iter = iter(test_dataloader)
                img = next(dataloader_iter)

            # Enviar datos a device tarjeta
            lr_img = img["lr"]
            hr_img = img["hr"]


            # 1. Forward pass
            test_pred = upscale_image(model=model,
                                      img=lr_img,
                                      upscale_factor=upscale_factor)

            # Busca forma de compararles con en el image original / alta calidad
            PSNR.append(psnr(hr_img, test_pred))
            SSIM.append(ssim(hr_img, test_pred))

            # 2. Calc loss/acc, adaptar a comparacion con PSNR y SSIM?
            loss = loss_fn(hr_img, test_pred)
            test_loss += loss.item()

        # Ajustar metrics y imprintir
        test_loss /= len(dataloader_iter)
        Avg_PSNR = mean(PSNR)
        Avg_SSIM = mean(SSIM)
    
    return test_results

# ?
def eval_model(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               device: torch.device = device):
    
    # Returns dictionary w/ results of model
    loss = 0
    model.eval()
    with torch.inference_mode():
        for batch, img in tqdm(enumerate(dataloader)):

            lr_img = img["lr"]
            hr_img = img["hr"]

            # Make predictions
            pred_output = upscale_image(model=model,
                                        img=lr_img,
                                        upscale_factor=upscale_factor)

            # Accumulate loss and acc values per batch
            loss += loss_fn(hr_img, pred_output)
            """acc += accuracy_fn(y_true=y,
                               y_pred=y_pred.argmax(dim=1))"""
            
        # Scale loss and acc to find avg loss/acc per batch
        loss /=  len(dataloader)

    return {"model_name": model.__class__.__name__, # works only when model was created with class)
            "model_loss": loss.item(),}


def run_training(model: torch.nn.Module,
                 train_dataloader: torch.utils.data.DataLoader,
                 test_dataloader: torch.utils.data.DataLoader,
                 optimiser: torch.optim.Optimizer,
                 loss_fn: torch.nn.Module,
                 epochs: int,
                 seed: None,
                 device = device):
    
    start_timer = timer()
    
    # 2. Crear diccionario de resultas vacia
    # # Devuelve el medio de los valores de un epoca
    results = {"train_loss": [],
               "test_loss": [],
               "PSNR_train_epoch": [],
               "SSIM_train_epoch": [],
               "PSNR_test_epoch": [],
               "SSIM_test_epoch": [],}
    
    PSNR_train_untrained = []
    SSIM_train_untrained = []
    PSNR_test_untrained = []
    SSIM_test_untrained = []

    # Crear dataloader separado por ese iteracion
    train_dataloader_untrained = iter(train_dataloader)

    # Se va atrapado aqui
    for i in range(train_batches):
        
        torch.cuda.empty_cache()

        try:
            train_img = next(train_dataloader_untrained)

        except StopIteration:
            dataloader_iter = iter(train_dataloader)
            train_img = next(train_dataloader_untrained)

        # print(train_img['hr'].shape)


        # print(train_img["lr"].shape)

        train_pred_img = upscale_image(model=model,
                                       img=train_img["lr"].to(device),
                                       upscale_factor=upscale_factor)

        # print(train_pred_img.shape)
        
        PSNR_train_untrained.append(psnr(train_img["hr"].to(device), torch.permute(train_pred_img, (1, 0, 2, 3)).to(device)))


        # Hace falta hacer en grayscale antes tomar el SSIM?
        SSIM_train_untrained.append(ssim(train_img["hr"].to(device), torch.permute(train_pred_img, (1, 0, 2, 3)).to(device)))

        print(torch.cuda.memory_allocated())

    # Crear dataloader separado por ese iteracion
    test_dataloader_untrained = iter(test_dataloader)

    for i in range(test_batches):

        torch.cuda.empty_cache()

        try:
            test_img = next(test_dataloader_untrained)

        except StopIteration:
            dataloader_iter = iter(test_dataloader)
            test_img = next(test_dataloader_untrained)

        test_pred_img = upscale_image(model=model,
                                      img=test_img["lr"].to(device), 
                                      upscale_factor=upscale_factor)

        PSNR_test_untrained.append(psnr(test_img["hr"].to(device), torch.permute(test_pred_img, (1, 0, 2, 3)).to(device)))

        # Hace falta hacer en grayscale antes tomar el SSIM?
        SSIM_test_untrained.append(ssim(test_img["hr"].to(device), torch.permute(test_pred_img, (1, 0, 2, 3)).to(device)))

        print(torch.cuda.memory_allocated())

    print("has logrado aqui")

    results["train_loss"].append(0)
    results["test_loss"].append(0)
    results["PSNR_train_epoch"].append(mean(PSNR_train_untrained))
    results["SSIM_train_epoch"].append(mean(SSIM_train_untrained))
    results["PSNR_test_epoch"].append(mean(PSNR_test_untrained))
    results["SSIM_test_epoch"].append(mean(SSIM_test_untrained))

    if seed:
        torch.cuda.manual_seed(seed)

    # 3. Hacer lazo a traves de entrenamiento
    for epoch in tqdm(range(epochs)):
        print(f"Epoch: {epoch}\n---------")

        print(torch.cuda.memory_allocated())

        train_dataloader_iter = iter(train_dataloader)

        test_dataloader_iter = iter(test_dataloader)
        
        train_results = train_step(model=model,
                                data_loader_iter=train_dataloader_iter,
                                loss_fn=loss_fn,
                                optimiser=optimiser,
                                device=device)
    
        test_results = test_step(model=model,
                              data_loader_iter=test_dataloader_iter,
                              loss_fn=loss_fn,
                              device=device)
        
        # Ponerle aqui?
        scheduler.step()

        # Adaptar a nuevos parametros de evaluacion
        # No lo se si podamos usarlo en convertirando verso train_step, test_step funcionas
        if epoch % 5 == 0:
            print(f"\nEpoch: {epoch} | Train loss: {train_results['train_loss']:.4f} | Test loss: {test_results['test_loss']:.4f}")
            print(model.state_dict())

        # 5. Poner al dia el diccionario de resultas
        results["train_loss"].append(train_results["train_loss"])
        results["test_loss"].append(test_results["test_loss"])
        results["PSNR_train_epoch"].append(train_results["PSNR"])
        results["SSIM_train_epoch"].append(train_results["SSIM"])
        results["PSNR_test_epoch"].append(test_results["PSNR"])
        results["SSIM_test_epoch"].append(test_results["SSIM"])

        end_timer = timer()
        total_train_time_model = print_train_time(start=start_timer, 
                                                  end=end_timer,
                                                  device=str(next(model.parameters()).device))
        print(total_train_time_model)

    # 6. Devuelve resultados 
    return results 

Averiguar el estado del modelo

In [356]:
print(model.parameters)

<bound method Module.parameters of ResolutionEnhancer(
  (conv_block_1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): GELU(approximate='none')
    (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
  )
  (conv_block_2): Sequential(
    (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
    (1): GELU(approximate='none')
    (2): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
    (3): GELU(approximate='none')
  )
  (output_layer): Sequential(
    (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
  )
)>


Ejecutar el entranimiento

In [357]:
epochs = 10

# Adaptar para este modelo
# Pienso que el accuracy function debe ser del PSNR y SSIM? O sea una comparacion con los PSNR y SSIM del original contra lo del predicion
model_results = run_training(model=model,
                             train_dataloader=train_dataloader,
                             test_dataloader=test_dataloader,
                             optimiser=optimiser,
                             loss_fn=loss_function,
                             epochs=epochs,
                             seed=None,
                             device=device)

torch.Size([32, 1, 448, 448])


ValueError: pic should be 2/3 dimensional. Got 4 dimensions.

Dataframe

In [ ]:
"""def PSNR(img1, img2):
    mse = np.mean(np.square(np.subtract(img1.astype(np.float64),
                                        img2.astype(np.float64))))
    if mse == 0:
        return np.Inf

    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX) - 10 * math.log10(mse)"""

results_df = pd.DataFrame(index=range(0, epochs), data=model_results)

print(results_df.head(5))

Definir functions para mostrar plots de resultados

In [ ]:
# Pude beneficiarnos a usar seaborn aqui
def plot_loss_curves(results: Dict[str, List[float]]):
    """Plots training curves of a results dictionary"""
    # Get loss values of results dict(train and test)
    train_loss = results["train_loss"]
    test_loss = results["test_loss"]
    PSNR_train = results["PSNR_train_epoch"]
    SSIM_train = results["SSIM_train_epoch"]
    PSNR_test = results["PSNR_test_epoch"]
    SSIM_test = results["SSIM_test_epoch"]

    # Establecer plot
    plot.figure(figsize=(15, 7))

    sns.set_theme(style="darkgrid")

    # Plot the responses for different events and regions
    sns.lineplot(x="epoch", y="train_loss",
                # hue="region", style="event",
                 data=train_loss)
    
    sns.lineplot(x="epoch", y="test_loss",
                # hue="region", style="event",
                 data=test_loss)
    
    sns.lineplot(x="epoch", y="PSNR_train",
                # hue="region", style="event",
                 data=PSNR_train)
    
    sns.lineplot(x="epoch", y="SSIM_train",
                # hue="region", style="event",
                 data=SSIM_train)
    
    sns.lineplot(x="epoch", y="PSNR_test",
                # hue="region", style="event",
                 data=PSNR_test)
    
    sns.lineplot(x="epoch", y="SSIM_test",
                # hue="region", style="event",
                 data=SSIM_test)


def pred_and_plot_image(model: torch.nn.Module,
                        image_path: str,
                        transform=None,
                        device=device):
    
    """Makes prediction on a target image w/ trained model and plots"""
    # Load image
    target_image = torchvision.io.read_image(str(image_path)).type(torch.float32).to(device)

    # Divide image pixel values by 255 to turn between [0, 1] ?
    target_image = target_image / 255.0

    # Transform if any
    if transform:
        target_image = transform(target_image)

    # Make sure the model is on target device
    model.to(device)

    with torch.inference_mode():
        # Add extra dimension to image (batch size dimension)
        target_image = target_image.unsqueeze(0)

        # Make predicition on image w/ extra dimension
        target_image_pred = model(target_image)
    
    PSNR = psnr(target_image_pred, target_image) # ?
    SSIM = ssim(target_image_pred, target_image) # ?
        
    plot.imshow(target_image.squeeze().permute(1, 2, 0))
    plot.imshow(target_image_pred)
    plot.title(f"PSNR: {PSNR} | SSIM: {SSIM}")
    plot.axis(False)
    

def predict_random_images(dataset: torch.utils.data.Dataset,
                          model: nn.Module,
                          n: int = 10,
                          display_shape: bool = True,
                          seed: int = None,
                          transform = None):
    
    # 2. Adjust display if n too high
    if n > 10:
        n = 10
        display_shape = False
        print(f"For display purposes, n shouldn't be larger than 10, setting to 10 and removing shape display.")
    
    # 3. Set random seed
    if seed:
        random.seed(seed)

    # 4. Get random sample indexes
    random_samples_idx = random.sample(range(len(dataset)), k=n)

    # 5. Setup plot
    plot.figure(figsize=(16, 8))

    # 6. Loop through samples and display random samples 
    for i, targ_sample in enumerate(random_samples_idx):

        target_image = torchvision.io.read_image(dataset[targ_sample]).type(torch.float32).to(device)

        # Divide image pixel values by 255 to turn between [0, 1]
        target_image = target_image / 255.0

        # Transform if any
        if transform:
            target_image = transform(target_image)

         # Make sure the model is on target device
        model.to(device)

    with torch.inference_mode():
        # Add extra dimension to image (batch size dimension)
        target_image = target_image.unsqueeze(0)

        # Make predicition on image w/ extra dimension
        target_image_pred = model(target_image)

        # 7. Adjust image tensor shape for plotting: [color_channels, height, width] -> [color_channels, height, width]
        # Porque permute no va a funcionar??
        torch.permute(target_image, (1, 2, 0))

    PSNR = psnr(target_image_pred, target_image) # ?
    SSIM = ssim(target_image_pred, target_image) # ?

    # Plot adjusted samples
    plot.subplot(1, n, i+1)
    plot.imshow(target_image)
    plot.imshow(target_image_pred)
    plot.title(f"PSNR: {PSNR} | SSIM: {SSIM}")
    plot.axis("off")

Plot curves

In [ ]:
plot_loss_curves(results=model_results)

predict_random_images(dataset=test_data,
                      model=model,
                      n=5,)

Guardar y Cargar Modelos

In [ ]:
# torch.save(model.state_dict(), PATH)
# torch.load()
# torch.nn.Module.model.load_state_dict()


MODEL_PATH = Path(r"D:\Documents\ImageEnhancer\Models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "resolution_enhancer.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# if MODEL_SAVE_PATH.is_dir():
torch.save(obj=model.state_dict(), f=MODEL_SAVE_PATH)
print(f"Saved model to {MODEL_SAVE_PATH}")


loaded_model = ResolutionEnhancer(input_channels=1,
                                  hidden_units=64,
                                  output_channels=1,
                                  upscale_factor=3) # Solo uno ejemplo

loaded_model.load_state_dict(torch.load(f=MODEL_SAVE_PATH))
